# QUADRATIC_ASSIGN

## Overview
The QUADRATIC_ASSIGN function solves the quadratic assignment problem (QAP), a classic combinatorial optimization problem in operations research. The QAP models the assignment of a set of facilities to a set of locations with the goal of minimizing the total cost, which is determined by the flow between facilities and the distance between locations. Mathematically, the objective is to find a permutation $\pi$ that minimizes:

```math
\text{Cost} = \sum_{i=1}^n \sum_{j=1}^n F_{i,j} \cdot D_{\pi(i),\pi(j)}
```

where $F$ is the flow matrix and $D$ is the distance matrix. This function uses `scipy.optimize.quadratic_assignment` to efficiently solve the QAP for small to medium-sized matrices. The QAP is widely used in facility layout, keyboard design, and other resource allocation problems.

## Usage
To use the QUADRATIC_ASSIGN function in Excel, enter it as a formula in a cell, specifying your flow and distance matrices as 2D lists or Excel ranges:

```excel
=QUADRATIC_ASSIGN(flow_matrix, distance_matrix)
```

## Arguments
| Argument        | Type     | Required | Description                                         | Example                |
|:---------------|:---------|:--------:|:----------------------------------------------------|:-----------------------|
| flow_matrix    | 2D list  | Yes      | The flow between facilities (square matrix).        | `{0,5,2;5,0,3;2,3,0}`  |
| distance_matrix| 2D list  | Yes      | The distances between locations (square matrix).    | `{0,2,3;2,0,1;3,1,0}`  |

## Returns
| Returns    | Type     | Description                                                      | Example                |
|:-----------|:---------|:-----------------------------------------------------------------|:-----------------------|
| Assignment | 2D list  | A 2D list of [facility, location] pairs representing the optimal assignment. | `[[0,2],[1,0],[2,1]]`    |
| Error      | string   | Error message if calculation fails                                | `"Both matrices must be square and of the same size."` |

## Examples

### Assigning Facilities to Minimize Cost
Assign 3 facilities to 3 locations to minimize total cost.

**Input:**
- Flow Matrix:
  | 0 | 5 | 2 |
  | 5 | 0 | 3 |
  | 2 | 3 | 0 |
- Distance Matrix:
  | 0 | 2 | 3 |
  | 2 | 0 | 1 |
  | 3 | 1 | 0 |

```excel
=QUADRATIC_ASSIGN(A1:C3, D1:F3)
```
**Output:**
```
[[0, 2], [1, 0], [2, 1]]
```

### Handling Invalid Input
If the matrices are not square or not the same size, an error message is returned.

**Input:**
- Flow Matrix:
  | 0 | 1 |
  | 1 | 0 |
- Distance Matrix:
  | 0 | 2 | 3 |
  | 2 | 0 | 1 |
  | 3 | 1 | 0 |

```excel
=QUADRATIC_ASSIGN(A1:B2, C1:E3)
```
**Output:**
```
"Both matrices must be square and of the same size."
```

In [ ]:
import numpy as np
from scipy.optimize import quadratic_assignment

def quadratic_assign(flow_matrix, distance_matrix):
    """
    Solves the quadratic assignment problem (QAP) for given flow and distance matrices.

    Args:
        flow_matrix (list): 2D list representing the flow between facilities.
        distance_matrix (list): 2D list representing the distances between locations.

    Returns:
        list: 2D list of [facility, location] assignments, or error message string.
    """
    try:
        flow = np.array(flow_matrix)
        dist = np.array(distance_matrix)
        if flow.ndim != 2 or dist.ndim != 2:
            return "Both inputs must be 2D lists (matrices)."
        if flow.shape != dist.shape or flow.shape[0] != flow.shape[1]:
            return "Both matrices must be square and of the same size."
        result = quadratic_assignment(flow, dist)
        if not hasattr(result, 'col_ind'):
            return str(result)
        assignment = [[int(i), int(j)] for i, j in enumerate(result.col_ind)]
        return assignment
    except Exception as e:
        return str(e)

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_basic_qap():
    flow_matrix = [
        [0, 5, 2],
        [5, 0, 3],
        [2, 3, 0]
    ]
    distance_matrix = [
        [0, 2, 3],
        [2, 0, 1],
        [3, 1, 0]
    ]
    result = quadratic_assign(flow_matrix, distance_matrix)
    assert isinstance(result, list)
    assert all(isinstance(pair, list) and len(pair) == 2 for pair in result)
    assert len(result) == 3

def test_invalid_input_not_2d():
    flow_matrix = [1, 2, 3]
    distance_matrix = [1, 2, 3]
    result = quadratic_assign(flow_matrix, distance_matrix)
    assert isinstance(result, str) and len(result) > 0

def test_invalid_shape():
    flow_matrix = [
        [0, 1],
        [1, 0]
    ]
    distance_matrix = [
        [0, 2, 3],
        [2, 0, 1],
        [3, 1, 0]
    ]
    result = quadratic_assign(flow_matrix, distance_matrix)
    assert isinstance(result, str) and len(result) > 0

def test_empty_matrix():
    flow_matrix = []
    distance_matrix = []
    result = quadratic_assign(flow_matrix, distance_matrix)
    assert isinstance(result, str) and len(result) > 0

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        [[0, 5, 2], [5, 0, 3], [2, 3, 0]],
        [[0, 2, 3], [2, 0, 1], [3, 1, 0]]
    ],
    [
        [[0, 1], [1, 0]],
        [[0, 2, 3], [2, 0, 1], [3, 1, 0]]
    ]
]

demo = gr.Interface(
    fn=quadratic_assign,
    inputs=[
        gr.Dataframe(headers=None, label="Flow Matrix", row_count=3, col_count=3, type="array",
                     value=[[0, 5, 2], [5, 0, 3], [2, 3, 0]]),
        gr.Dataframe(headers=None, label="Distance Matrix", row_count=3, col_count=3, type="array", 
                     value=[[0, 2, 3], [2, 0, 1], [3, 1, 0]]),
    ],
    outputs=gr.Dataframe(headers=["Facility", "Location"], label="Assignments"),
    examples=examples,
    description="Solve the Quadratic Assignment Problem (QAP) by providing flow and distance matrices. The output is a list of [facility, location] assignments. If the input is invalid, an error message is returned.",
    flagging_mode="never",
)
demo.launch()